In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

## 数据准备

In [2]:
datapath=r'/Users/nilei/Nutstore/Data/EURUSD_D1.csv'         #macos
dt=pd.read_csv(datapath)
dt.rename(columns={'Time (UTC)':'Date'},inplace=True)
dt.index=pd.to_datetime(dt.Date)
dt.drop(columns='Date',inplace=True)
dt.head()

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-01,1.43283,1.43356,1.43181,1.43335,76787.4
2010-01-03,1.43295,1.43359,1.42910,1.43141,6331.3
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.4
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.1
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.8


In [3]:
dt.rename(columns={'Open':'open','High':'high','Low':'low','Close':'close','Volume ':'volume'},inplace=True)

In [4]:
dt.head()

,open,high,low,close,volume
Date,,,,,
2010-01-01,1.43283,1.43356,1.43181,1.43335,76787.4
2010-01-03,1.43295,1.43359,1.42910,1.43141,6331.3
2010-01-04,1.43143,1.44556,1.42559,1.44244,80019.4
2010-01-05,1.44238,1.44834,1.43445,1.43634,79887.1
2010-01-06,1.43638,1.44342,1.42807,1.44005,80971.8


In [5]:
import datetime as datetime
data = bt.feeds.PandasData(dataname=dt,                               
                            fromdate=datetime.datetime(2012, 1, 1),                               
                            todate=datetime.datetime(2020, 1, 1) )           #这里面要尤其注意

## 策略细节

In [6]:
class TurtleStrategy(bt.Strategy):
    params = (('long_period',20),
              ('short_period',10),  
              ('printlog', False), )   

    def __init__(self):        

        self.order = None      
        self.buyprice = 0      
        self.buycomm = 0      
        self.buy_size = 0      
        self.buy_count = 0       
        # 海龟交易法则中的唐奇安通道和平均波幅ATR        
        self.H_line = bt.indicators.Highest(self.data.high(-1), period=self.p.long_period)        
        self.L_line = bt.indicators.Lowest(self.data.low(-1), period=self.p.short_period)       
        self.TR = bt.indicators.Max((self.data.high(0)- self.data.low(0)),\
                                    abs(self.data.close(-1)-self.data.high(0)), \
                                    abs(self.data.close(-1)  - self.data.low(0)))        
        self.ATR = bt.indicators.SimpleMovingAverage(self.TR, period=14)       
        # 价格与上下轨线的交叉      
        self.buy_signal = bt.ind.CrossOver(self.data.close(0), self.H_line)        
        self.sell_signal = bt.ind.CrossOver(self.data.close(0), self.L_line)    
    
    def next(self): 
        if self.order:
            return        
        #入场：价格突破上轨线且空仓时        
        if self.buy_signal > 0 and self.buy_count == 0:                                 
            self.buy_size = self.broker.getvalue() * 0.01 / self.ATR            
            self.buy_size  = int(self.buy_size  / 100) * 100                             
            self.sizer.p.stake = self.buy_size             
            self.buy_count = 1            
            self.order = self.buy()        
        #加仓：价格上涨了买入价的0.5的ATR且加仓次数少于3次（含）        
        elif self.data.close >self.buyprice+0.5*self.ATR[0] and self.buy_count > 0 and self.buy_count <=4:           
            self.buy_size  = self.broker.getvalue() * 0.01 / self.ATR            
            self.buy_size  = int(self.buy_size  / 100) * 100            
            self.sizer.p.stake = self.buy_size             
            self.order = self.buy()           
            self.buy_count += 1        
        #离场：价格跌破下轨线且持仓时        
        elif self.sell_signal < 0 and self.buy_count > 0:            
            self.order = self.sell()            
            self.buy_count = 0        
        #止损：价格跌破买入价的2个ATR且持仓时        
        elif self.data.close < (self.buyprice - 2*self.ATR[0]) and self.buy_count > 0:           
            self.order = self.sell()
            self.buy_count = 0   
            
    #交易记录日志（默认不打印结果）
    def log(self, txt, dt=None,doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')
    
    #记录交易执行情况（默认不输出结果）
    def notify_order(self, order):
        # 如果order为submitted/accepted,返回空
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 如果order为buy/sell executed,报告价格结果
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log(f'买入:\n价格:{order.executed.price},\
                成本:{order.executed.value},\
                手续费:{order.executed.comm}')
    
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'卖出:\n价格：{order.executed.price},\
                成本: {order.executed.value},\
                手续费{order.executed.comm}')
            
            self.bar_executed = len(self) 
            
        # 如果指令取消/交易失败, 报告结果
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失败')
        self.order = None
    
    #记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'策略收益：\n毛收益 {trade.pnl:.2f}, 净收益 {trade.pnlcomm:.2f}')
    
    def stop(self):
        self.log(f'(组合线：{self.p.long_period},{self.p.short_period})； \
        期末总资金: {self.broker.getvalue():.2f}', doprint=True)

In [7]:
class TradeSizer(bt.Sizer):
    params = (('stake', 1),)    
    def _getsizing(self, comminfo, cash, data, isbuy):        
        if isbuy:          
            return self.p.stake        
        position = self.broker.getposition(data)        
        if not position.size:            
            return 0        
        else:            
            return position.size        
        return self.p.stake

## 回测框架

In [12]:
def main(data,long_list,short_list,startcash=1000000,com=0.001):
    #创建主控制器
    cerebro = bt.Cerebro()      
    #导入策略参数寻优
    cerebro.optstrategy(TurtleStrategy,long_period=long_list,short_period=short_list)    
    #将数据加载至回测系统    
    cerebro.adddata(data)
    #broker设置资金、手续费
    cerebro.broker.setcash(startcash)           
    cerebro.broker.setcommission(commission=com)    
    #设置买入设置，策略，数量
    cerebro.addsizer(TradeSizer)    
    print('期初总资金: %15.2f' % cerebro.broker.getvalue())    
    cerebro.run(maxcpus=1)    

## 参数寻优

In [13]:
long_list=range(20,70,5)
short_list=range(5,20,5)
main(data,long_list,short_list)

期初总资金:      1000000.00
2020-01-01,(组合线：20,5)；         期末总资金: 975018.42
2020-01-01,(组合线：20,10)；         期末总资金: 964020.00
2020-01-01,(组合线：20,15)；         期末总资金: 956279.30
2020-01-01,(组合线：25,5)；         期末总资金: 986672.28
2020-01-01,(组合线：25,10)；         期末总资金: 981626.45
2020-01-01,(组合线：25,15)；         期末总资金: 973745.79
2020-01-01,(组合线：30,5)；         期末总资金: 974303.75
2020-01-01,(组合线：30,10)；         期末总资金: 971092.97
2020-01-01,(组合线：30,15)；         期末总资金: 963296.02
2020-01-01,(组合线：35,5)；         期末总资金: 974303.75
2020-01-01,(组合线：35,10)；         期末总资金: 971092.97
2020-01-01,(组合线：35,15)；         期末总资金: 963296.02
2020-01-01,(组合线：40,5)；         期末总资金: 974303.75
2020-01-01,(组合线：40,10)；         期末总资金: 971092.97
2020-01-01,(组合线：40,15)；         期末总资金: 963296.02
2020-01-01,(组合线：45,5)；         期末总资金: 974303.75
2020-01-01,(组合线：45,10)；         期末总资金: 971092.97
2020-01-01,(组合线：45,15)；         期末总资金: 963296.02
2020-01-01,(组合线：50,5)；         期末总资金: 956815.43
2020-01-01,(组合线：50,10)；         期末总资金: 953655.10
2020

怎么样找出最大值

## 策略封装

In [13]:
def performance(TurtleStrategy,data,long,short,startcash,com):
    cerebro = bt.Cerebro()      
    #导入策略参数寻优
    cerebro.addstrategy(TurtleStrategy,long_period=long,short_period=short)    
    #将数据加载至回测系统
    cerebro.adddata(data)
    #broker设置资金、手续费
    cerebro.broker.setcash(startcash)           
    cerebro.broker.setcommission(commission=com)    
    #设置买入设置，策略，数量
    cerebro.addsizer(TradeSizer)
    #df00,df0,df1,df2,df3,df4=bt.out_result(TurtleStrategy,data)
    print('期初总资金: %.2f' % cerebro.broker.getvalue())    
    cerebro.run() 

In [14]:
from collections import OrderedDict
def performance2(TurtleStrategy,data,long,short,startcash,com):
    cerebro = bt.Cerebro()      
    #导入策略参数寻优
    cerebro.addstrategy(TurtleStrategy,long_period=long,short_period=short)    
    #将数据加载至回测系统
    cerebro.adddata(data)
    #broker设置资金、手续费
    cerebro.broker.setcash(startcash)           
    cerebro.broker.setcommission(commission=com)    
    #设置买入设置，策略，数量
    cerebro.addsizer(TradeSizer)
        
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.TotalValue, _name='_TotalValue')
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn')
    cerebro.addanalyzer(bt.analyzers.Calmar, _name='_Calmar')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown')
    cerebro.addanalyzer(bt.analyzers.TimeDrawDown, _name='_TimeDrawDown')
    cerebro.addanalyzer(bt.analyzers.GrossLeverage, _name='_GrossLeverage')
    cerebro.addanalyzer(bt.analyzers.PositionsValue, _name='_PositionsValue')
    cerebro.addanalyzer(bt.analyzers.LogReturnsRolling, _name='_LogReturnsRolling')
    cerebro.addanalyzer(bt.analyzers.PeriodStats, _name='_PeriodStats')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='_Returns')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='_SharpeRatio_A')
    cerebro.addanalyzer(bt.analyzers.SQN, _name='_SQN')
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='_TimeReturn')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='_TradeAnalyzer')
    cerebro.addanalyzer(bt.analyzers.Transactions, _name='_Transactions')
    cerebro.addanalyzer(bt.analyzers.VWR, _name='_VWR')
    
    results = cerebro.run()
    
    performance_dict=OrderedDict()
    calmar_ratio=list(results[0].analyzers._Calmar.get_analysis().values())[-1]
    drawdown_info=results[0].analyzers._DrawDown.get_analysis()
    average_drawdown_len=drawdown_info['len']
    average_drawdown_rate=drawdown_info['drawdown']
    average_drawdown_money=drawdown_info['moneydown']
    max_drawdown_len=drawdown_info['max']['len']
    max_drawdown_rate=drawdown_info['max']['drawdown']
    max_drawdown_money=drawdown_info['max']['moneydown']
    PeriodStats_info=results[0].analyzers._PeriodStats.get_analysis()
    average_rate=PeriodStats_info['average']
    stddev_rate=PeriodStats_info['stddev']
    positive_year=PeriodStats_info['positive']
    negative_year=PeriodStats_info['negative']
    nochange_year=PeriodStats_info['nochange']
    best_year=PeriodStats_info['best']
    worst_year=PeriodStats_info['worst']
    SQN_info=results[0].analyzers._SQN.get_analysis()
    sqn_ratio=SQN_info['sqn']
    VWR_info=results[0].analyzers._VWR.get_analysis()
    vwr_ratio=VWR_info['vwr']
    sharpe_info=results[0].analyzers._SharpeRatio.get_analysis()
    # sharpe_info=results[0].analyzers._SharpeRatio_A.get_analysis()
    sharpe_ratio=sharpe_info['sharperatio']

    performance_dict['calmar_ratio']=round(calmar_ratio*100,3)
    performance_dict['average_drawdown_len']=average_drawdown_len
    performance_dict['average_drawdown_rate']=round(average_drawdown_rate,2)
    performance_dict['average_drawdown_money']=round(average_drawdown_money,2)
    performance_dict['max_drawdown_len']=max_drawdown_len
    performance_dict['max_drawdown_rate']=round(max_drawdown_rate,2)
    performance_dict['max_drawdown_money']=round(max_drawdown_money,2)
    performance_dict['average_rate']=round(average_rate,2)
    performance_dict['stddev_rate']=round(stddev_rate,2)
    performance_dict['positive_year']=positive_year
    performance_dict['negative_year']=negative_year
    performance_dict['nochange_year']=nochange_year
    performance_dict['best_year']=round(best_year*100,2)
    performance_dict['worst_year']=round(worst_year*100,2)
    performance_dict['sqn_ratio']=round(sqn_ratio,2)
    performance_dict['vwr_ratio']=round(vwr_ratio,2)
    performance_dict['sharpe_info']=round(sharpe_ratio,2)
    performance_dict['omega']=0

    trade_dict_1=OrderedDict()
    trade_dict_2=OrderedDict()
    trade_info=results[0].analyzers._TradeAnalyzer.get_analysis()
    total_trade_num=trade_info['total']['total']
    total_trade_opened=trade_info['total']['open']
    total_trade_closed=trade_info['total']['closed']
    total_trade_len=trade_info['len']['total']
    long_trade_len=trade_info['len']['long']['total']
    short_trade_len=trade_info['len']['short']['total']
    
    longest_win_num=trade_info['streak']['won']['longest']
    longest_lost_num=trade_info['streak']['lost']['longest']
    net_total_pnl=trade_info['pnl']['net']['total']
    net_average_pnl=trade_info['pnl']['net']['average']
    win_num=trade_info['won']['total']
    win_total_pnl=trade_info['won']['pnl']['total']
    win_average_pnl=trade_info['won']['pnl']['average']
    win_max_pnl=trade_info['won']['pnl']['max']
    lost_num=trade_info['lost']['total']
    lost_total_pnl=trade_info['lost']['pnl']['total']
    lost_average_pnl=trade_info['lost']['pnl']['average']
    lost_max_pnl=trade_info['lost']['pnl']['max']
    
    trade_dict_1['total_trade_num']=total_trade_num
    trade_dict_1['total_trade_opened']=total_trade_opened
    trade_dict_1['total_trade_closed']=total_trade_closed
    trade_dict_1['total_trade_len']=total_trade_len
    trade_dict_1['long_trade_len']=long_trade_len
    trade_dict_1['short_trade_len']=short_trade_len
    trade_dict_1['longest_win_num']=longest_win_num
    trade_dict_1['longest_lost_num']=longest_lost_num
    trade_dict_1['net_total_pnl']=net_total_pnl
    trade_dict_1['net_average_pnl']=net_average_pnl
    trade_dict_1['win_num']=win_num
    trade_dict_1['win_total_pnl']=win_total_pnl
    trade_dict_1['win_average_pnl']=win_average_pnl
    trade_dict_1['win_max_pnl']=win_max_pnl
    trade_dict_1['lost_num']=lost_num
    trade_dict_1['lost_total_pnl']=lost_total_pnl
    trade_dict_1['lost_average_pnl']=lost_average_pnl
    trade_dict_1['lost_max_pnl']=lost_max_pnl
    
    long_num=trade_info['long']['total']
    long_win_num=trade_info['long']['won']
    long_lost_num=trade_info['long']['lost']
    long_total_pnl=trade_info['long']['pnl']['total']
    long_average_pnl=trade_info['long']['pnl']['average']
    long_win_total_pnl=trade_info['long']['pnl']['won']['total']
    long_win_max_pnl=trade_info['long']['pnl']['won']['max']
    long_lost_total_pnl=trade_info['long']['pnl']['lost']['total']
    long_lost_max_pnl=trade_info['long']['pnl']['lost']['max']
    
    short_num=trade_info['short']['total']
    short_win_num=trade_info['short']['won']
    short_lost_num=trade_info['short']['lost']
    short_total_pnl=trade_info['short']['pnl']['total']
    short_average_pnl=trade_info['short']['pnl']['average']
    short_win_total_pnl=trade_info['short']['pnl']['won']['total']
    short_win_max_pnl=trade_info['short']['pnl']['won']['max']
    short_lost_total_pnl=trade_info['short']['pnl']['lost']['total']
    short_lost_max_pnl=trade_info['short']['pnl']['lost']['max']
    
    trade_dict_2['long_num']=long_num
    trade_dict_2['long_win_num']=long_win_num
    trade_dict_2['long_lost_num']=long_lost_num
    trade_dict_2['long_total_pnl']=long_total_pnl
    trade_dict_2['long_average_pnl']=long_average_pnl
    trade_dict_2['long_win_total_pnl']=long_win_total_pnl
    trade_dict_2['long_win_max_pnl']=long_win_max_pnl
    trade_dict_2['long_lost_total_pnl']=long_lost_total_pnl
    trade_dict_2['long_lost_max_pnl']=long_lost_max_pnl
    trade_dict_2['short_num']=short_num
    trade_dict_2['short_win_num']=short_win_num
    trade_dict_2['short_lost_num']=short_lost_num
    trade_dict_2['short_total_pnl']=short_total_pnl
    trade_dict_2['short_average_pnl']=short_average_pnl
    trade_dict_2['short_win_total_pnl']=short_win_total_pnl
    trade_dict_2['short_win_max_pnl']=short_win_max_pnl
    trade_dict_2['short_lost_total_pnl']=short_lost_total_pnl
    trade_dict_2['short_lost_max_pnl']=short_lost_max_pnl

    df00=pd.DataFrame(index=range(len(performance_dict)))
    df01=pd.DataFrame([performance_dict]).T
    df01.columns=['绩效指标值']
    df02=pd.DataFrame([trade_dict_1]).T
    df02.columns=['普通交易指标值']
    df03=pd.DataFrame([trade_dict_2]).T
    df03.columns=['多空交易指标值']
    df00['绩效指标']=df01.index
    df00['绩效指标值']=df01.round(4).values
    df00['普通交易指标']=df02.index
    df00['普通交易指标值']=[round(float(i),4) for i in list(df02['普通交易指标值'])]
    df00['多空交易指标']=df03.index
    df00['多空交易指标值']=[round(float(i),4) for i in list(df03['多空交易指标值'])]
    
    # 账户收益率
    df0=df1=pd.DataFrame([results[0].analyzers._TotalValue.get_analysis()]).T
    df0.columns=['total_value']
    
    # 总的杠杆
    df1=pd.DataFrame([results[0].analyzers._GrossLeverage.get_analysis()]).T
    df1.columns=['GrossLeverage']
    

    # 滚动的对数收益率
    df2=pd.DataFrame([results[0].analyzers._LogReturnsRolling.get_analysis()]).T
    df2.columns=['log_return']
    
    # year_rate
    df3=pd.DataFrame([results[0].analyzers._AnnualReturn.get_analysis()]).T
    df3.columns=['year_rate']
    
    # 总的持仓价值
    df4=pd.DataFrame(results[0].analyzers._PositionsValue.get_analysis()).T
    df4['total_position_value']=df4.sum(axis=1)

    pyfoliozer = results[0].analyzers.getbyname('pyfolio')
    returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    return df00,df0,df1,df2,df3,df4

In [15]:
k1,k2,k3,k4,k5,k6=performance2(TurtleStrategy,data,20,10,1000000,0.001)

2020-01-01,(组合线：20,10)；         期末总资金: 964020.00


In [16]:
k1

,绩效指标,绩效指标值,普通交易指标,普通交易指标值,多空交易指标,多空交易指标值
0,calmar_ratio,0.00,total_trade_num,3.0000,long_num,3.0000
1,average_drawdown_len,1448.00,total_trade_opened,0.0000,long_win_num,0.0000
2,average_drawdown_rate,4.35,total_trade_closed,3.0000,long_lost_num,3.0000
3,average_drawdown_money,43824.62,total_trade_len,33.0000,long_total_pnl,-35979.9981
4,max_drawdown_len,1448.00,long_trade_len,33.0000,long_average_pnl,-11993.3327
5,max_drawdown_rate,4.35,short_trade_len,0.0000,long_win_total_pnl,0.0000
6,max_drawdown_money,43824.62,longest_win_num,0.0000,long_win_max_pnl,0.0000
7,average_rate,-0.00,longest_lost_num,3.0000,long_lost_total_pnl,-35979.9981
8,stddev_rate,0.01,net_total_pnl,-35979.9981,long_lost_max_pnl,-17937.0265
9,positive_year,0.00,net_average_pnl,-11993.3327,short_num,0.0000
